#获取数据

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.sans-serif'] = 'SimHei'

In [ ]:
LC = pd.read_csv('Lc.csv')

#数据清洗

In [ ]:
#通过info观测到数据集只有一个字段类型有问题
LC.info()

In [ ]:
#把时间类型处理
LC['借款成功日期'] = pd.to_datetime(LC['借款成功日期'])

In [ ]:
LC.describe([.25,.50,.75,.99]).T

#1.观察年龄分布，最小的17岁，平均年龄29岁，33岁以下的人群占比超过75%，说明中青年较多
2.观察借款分布，最小的借款金额为100元，最大值50万元，平均值4424元，低于5230的借款金额占到了75%，说明小额贷款较多

#用户画像

In [ ]:
#性别分析
LC['性别'].value_counts()/LC['性别'].value_counts().sum()
#男生在我们平台贷款的占比高达67.6%
LC['性别'].value_counts().plot(kind='pie',autopct='%.1f%%')
male = LC.loc[Lc['性别']=='男']
fmale = LC.loc[Lc['性别']!='男']
plt.subplot(121)
date = [male['借款金额'].sum(),fmale['借款金额'].sum()]
_ = plt.pie(date,labels=['男'，'女']，autopct='%.1f%%')
#男士在我们平台总贷款金额的占比高代69%,但是男士的平均贷款金额仅比女士多一点（2%），可以看出男女贷款金额需求没有太大区别
plt.subplot(122)
date =[male['借款金额'].sum/male['借款金额'].count(),fmale['借款金额'].sum/fmale['借款金额'].count()]
_ = plt.pie(date,labels=['男'，'女']，autopct='%.1f%%')

In [ ]:
#新老客户分析
plt.subplot（121)
LC.groupby('是否首标')['ListingId'].count().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客人数占比')
plt.subplot(121)
LC.groupby('是否首标')['借款金额'].sum().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客金额占比')

In [ ]:
(LC.groupby('是否首标')['借款金额'].sum()/LC.groupby('是否首标')['ListingId'].count()).plot(kind='pie',autopct='%.1f%%')

#老客户占比73%，说明平台的粘性比较高，并且新客平均借款水平要远高于老客（将近1700元的水平）

In [ ]:
plt.subplot(221)
fmale.groupby('是否首标')['ListingId'].count().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客人数占比')
plt.subplot(222)
fmale.groupby('是否首标')['借款金额'].sum().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客金额占比')
plt.subplot(223)
male.groupby('是否首标')['ListingId'].count().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客人数占比')
plt.subplot(224)
male.groupby('是否首标')['借款金额'].sum().plot(kind='pie',autopct='%.1f%%')
plt.title('新老客金额占比')

In [ ]:
new = LC.loc[LC['是否首标'] =='是']
old = LC.loc[LC['是否首标'] == '否']
plot.subplot(121)
new.groupby('性别')['ListingId'].count().plot(kind='pie',autopct='%.1f%%')
plot.subplot(122)
new.groupby('性别')['借款金额'].sum().plot(kind='pie',autopct='%.1f%%')

#发现新客更多为男性，但从新客角度出发去看性别，发现性别对新客影响不大

#年龄分析

In [ ]:
bins = [15,20,25,30,35,40,60]
(pd.cut(LC['年龄'],bins=bins).value_counts()/pd.cut(LC['年龄'],bins=bins).value_counts().sum()).sort_index().plot(kind='bar')
LC['年龄区间'] = pd.cut(LC['年龄'],bins=bins)

In [ ]:
Lc.pivot_table(index='年龄区间',columns='是否首标',values='借款金额',aggfunc=np.size).plot

In [ ]:
plt.subplot(121)
(new.groupby('年龄区间')['ListingId'].count()/new.groupby('年龄区间')['ListingId'].count().sum()).sort_index().plot(kind='bar')
(old.groupby('年龄区间')['ListingId'].count()/old.groupby('年龄区间')['ListingId'].count().sum()).sort_index().plot(kind='bar')

In [ ]:
new.groupby('年龄区间')['借款金额'].sum()/new.groupby('年龄区间')['ListingId'].count()
new.groupby('年龄区间')['借款金额'].sum()/new.groupby('年龄区间')['ListingId'].count()/new.groupby('年龄区间')['借款金额'].sum()/(new.groupby('年龄区间')['ListingId'].count()
old.groupby('年龄区间')['借款金额'].sum()/old.groupby('年龄区间')['ListingId'].count()).sum()
old.groupby('年龄区间')['借款金额'].sum()/old.groupby('年龄区间')['ListingId'].count()/old.groupby('年龄区间')['借款金额'].sum()/old.groupby('年龄区间')['ListingId'].count().sum()


#新客的平均消费水平是5686，20-25之间的新客多一点，但是15-30的新客更有潜力，因为他们的平均贷款水平高于老客

#资金储备

In [ ]:
save_money = LC[['借款成功日期','借款金额']].copy
save_money.groupby('借款成功日期')['借款金额'].sum().sot_index.plot()
save_money.set_index('借款成功日期').resample('M')['借款金额'].sum().plot()
temp = save_money.set_index('借款成功日期').loc['2017-01'].reset_index().groupby('借款成功日期')['借款金额'].sum()
temp.mean() - 3*temp.std(),temp.mean() + 3*temp.std()
#每日准备的资金为11814846

#逾期率分析

In [ ]:
overdue = LC[['借款金额'，'初始评级','借款类型','性别','年龄','历史正常还款期数','历史逾期还款期数']]
overdue.loc[:,'逾期率'] = overdue['历史逾期还款期数']/(overdue['历史逾期还款期数']+overdue['历史正常还款期数'])
overdue.groupby('初始评级')['逾期率'].mean().plot()
#ef的表现可能是因为数据量过少，ef的平均逾期率呈断崖式上升
overdue.groupby('借款类型')['逾期率'].mean().plot()
#app闪电的逾期率明显低于其他三个，所以建议公司与app闪电合作
overdue.groupby('性别')['逾期率'].mean().plot(kind='bar')
bins=[0,2000,3000,4000,5000,6000,7000,8000,9000,10000,np.inf]
overdue['借款区间'] = pd.cut(overdue['借款金额'],bins=bins)
overdue.groupby('借款区间')['逾期率'].mean().plot(kind='bar')
#借款金额在2000以下的逾期率最低，2000-3000逾期率最高，可以考虑促进小额贷款的营销模式

#借款利率分析

In [ ]:
LC.groupby('借款利率')['借款金额'].count().sort_index()
LC['利率评级']= LC['借款利率'].apply(lambda x:'较高' if x >20 else '较低')
LC.groupby(['利率评级','性别'])['借款金额'].count().unstack()/LC.groupby(['利率评级','性别'])['借款金额'].count().unstack().sum()
#女性所能接受的贷款利率低于男性，而且女性的逾期率也高于男性，所以我们可以针对女性定制产品，例如：分笔贷款，降低利率